In [1]:
import rasterio
import numpy as np

# Path to your GeoTIFF
tif_path = "../tsa41/inventory_init.tif"
N = 10  # size of the square you want to extract

with rasterio.open(tif_path) as src:
    # Get image dimensions
    height, width = src.height, src.width
    bands = src.count

    # Compute the center of the image
    center_x = width // 2
    center_y = height // 2

    # Calculate window bounds (ensure even size centered around center)
    half_N = N // 2
    row_start = max(center_y - half_N, 0)
    row_end   = min(center_y + half_N, height)
    col_start = max(center_x - half_N, 0)
    col_end   = min(center_x + half_N, width)

    # Define the window
    window = rasterio.windows.Window(col_start, row_start, col_end - col_start, row_end - row_start)

    # Read the window from all bands
    data = src.read(window=window)  # shape: (bands, N, N)

# `data` is now a 3D numpy array of shape (bands, N, N)
print(data.shape)

(3, 10, 10)


In [3]:
import rasterio
from rasterio.windows import Window
from rasterio.transform import Affine

tif_path = "../tsa41/inventory_init.tif"
out_path = "tsa41mini-100x100.tif"
N = 100

with rasterio.open(tif_path) as src:
    height, width = src.height, src.width
    bands = src.count
    dtype = src.dtypes[0]

    center_x = width // 2
    center_y = height // 2
    half_N = N // 2

    row_start = max(center_y - half_N, 0)
    row_end   = min(center_y + half_N, height)
    col_start = max(center_x - half_N, 0)
    col_end   = min(center_x + half_N, width)

    # Define the window
    window = Window(col_start, row_start, col_end - col_start, row_end - row_start)

    # Read data from the window
    data = src.read(window=window)

    # Update the transform to reflect the window position
    transform = src.window_transform(window)

    # Prepare metadata for the new file
    meta = src.meta.copy()
    meta.update({
        "height": data.shape[1],
        "width": data.shape[2],
        "transform": transform
    })

    # Write out the cropped image
    with rasterio.open(out_path, "w", **meta) as dst:
        dst.write(data)

print(f"Saved cropped GeoTIFF to {out_path}")

Saved cropped GeoTIFF to tsa41mini-100x100.tif
